In [7]:
import os
cwd =  os.getcwd().replace("notebooks","")
os.chdir(cwd)

# WDD-only (belt & suspenders)
os.environ["MICROLM_DISABLE_SHIM"]   = "1"
os.environ["MICROLM_DISABLE_MAPPER"] = "1"

In [8]:
import os, sys, warnings
sys.path.insert(0, os.path.join(os.getcwd(), "src"))
warnings.filterwarnings("ignore", category=FutureWarning)

from micro_lm.compat.legacy_imports import install as _ml_legacy_install
_ml_legacy_install()  # aliases legacy 'encoders' & exports SBERTEncoder into __main__

In [9]:
from micro_lm.domains.arc.families_wdd import arc_family_registry

K = 12  # try 12 first; we’ll adjust if needed
fams = arc_family_registry(K=K)
print("families:", len(fams))
print([(f.name, f.idxs[:5]) for f in fams[:6]])
max_idx = max([i for f in fams for i in f.idxs], default=-1)
print("max idx in registry:", max_idx, "   K:", K)

families: 8
[('rotate', [0, 1, 2, 3]), ('flip', [4, 5]), ('color_map', [6]), ('translate', [7]), ('crop_pad', [8]), ('tile', [9])]
max idx in registry: 11    K: 12


In [10]:
from micro_lm.core.audit import run_families

policy = {
  "audit": {
    "backend":"wdd","mode":"family","K":K,
    "template_width":64,
    "z_abs": 0.0,      # ← relax
    "keep_frac": 0.0,  # ← relax
    "overrides": {}    # you can also tune per-family here by name
  },
  "mapper": {"use_shim_fallback": False, "confidence_threshold": 1.01, "model_path": ""},
}

def probe(p):
    res = run_families(p, fams, policy, {})
    print(p, "->", res.get("order"), "| keep:", res.get("keep"))
    return res

probe("flip the grid vertically")
probe("rotate the grid 90 degrees, then flip the grid vertically")


flip the grid vertically -> [] | keep: ['rotate', 'flip', 'color_map', 'translate', 'crop_pad', 'tile', 'rot90', 'cc_mask_largest']
rotate the grid 90 degrees, then flip the grid vertically -> [] | keep: ['rotate', 'flip', 'color_map', 'translate', 'crop_pad', 'tile', 'rot90', 'cc_mask_largest']


{'order': [],
 'keep': ['rotate',
  'flip',
  'color_map',
  'translate',
  'crop_pad',
  'tile',
  'rot90',
  'cc_mask_largest'],
 'route': ['defi', 'wdd', 'family'],
 'reason': 'audit.facade_stub'}

In [3]:
# 2) Import the family runner + ARC registry (mirrors your DeFi setup)
from micro_lm.core.audit import run_families
from micro_lm.domains.arc.families_wdd import arc_family_registry  # <- your ARC tooling

# 3) Strict WDD policy (same knobs you used for DeFi)
policy = {
    "audit": {"backend":"wdd","mode":"family","K":12,"template_width":64,"z_abs":0.55,"keep_frac":0.70},
    # mapper fallback stays OFF for Tier-2 strict tests
    "mapper": {"use_shim_fallback": False, "confidence_threshold": 1.01, "model_path": ""},
}

# 4) Build ARC families & probe a couple of “exec” prompts
fams = arc_family_registry(K=policy["audit"]["K"])
print(run_families("create a calendar event tomorrow at 3pm titled team sync", fams, policy, {}).get("order"))
print(run_families("send a summary email to the project list",                   fams, policy, {}).get("order"))

[]
[]


In [11]:
x = {}
x['a'] = 1
x.get('a')

1

In [ ]:
# from micro_lm import run_micro
# from micro_lm.domains.arc.families_wdd import arc_family_registry  # your ARC registry module

# STRICT_POLICY = {
#     "audit": {
#         "backend": "wdd",
#         "mode": "family",
#         "K": 12, "template_width": 64, "z_abs": 0.55, "keep_frac": 0.70,  # tune as needed
#     },
#     # add ARC rails here if you have them (limits, safe ops, etc.)
#     "mapper": {  # block legacy fallback
#         "use_shim_fallback": False,
#         "confidence_threshold": 1.01,
#         "model_path": ""
#     },
# }
# STRICT_CONTEXT = {
#     # ARC context (accounts/services) if your runner expects any; otherwise leave {}
# }

# # Sanity: registry builds cleanly
# _ = arc_family_registry(K=STRICT_POLICY["audit"]["K"])


In [ ]:
# import json

# def run_arc_strict(prompt: str, *, T=180, rails="stage11"):
#     out = run_micro(domain="arc", prompt=prompt,
#                     context=dict(STRICT_CONTEXT),
#                     policy=json.loads(json.dumps(STRICT_POLICY)),
#                     rails=rails, T=T)
#     # hard fail if any shim/mapper snuck in
#     v = (out.get("verify") or {})
#     arts = (out.get("artifacts") or {})
#     flags = (out.get("flags") or {})
#     mapper_in_arts = ("mapper" in arts) or ("artifacts" in arts and isinstance(arts["artifacts"], dict) and "mapper" in arts["artifacts"])
#     if flags.get("mapper_fallback") or "shim:" in (v.get("reason","").lower()) or mapper_in_arts:
#         raise AssertionError(f"WDD-only violation for {prompt!r}: reason={v.get('reason')}, flags={flags}, arts_keys={list(arts.keys())}")
#     seq = (out.get("plan") or {}).get("sequence") or []
#     if not seq:
#         raise AssertionError(f"No plan.sequence produced for {prompt!r} (WDD family mode).")
#     return out


In [ ]:
# print("=== ARC exec ===")
# ARC_EXEC = [
#     "create a calendar event tomorrow at 3pm titled team sync",
#     "send a summary email to the project list",
#     "add a todo: follow up with Alice on budget",
# ]
# for p in ARC_EXEC:
#     o = run_arc_strict(p)
#     print(p, "→", (o.get("plan") or {}).get("sequence"))

# print("\n=== ARC edges (should be blocked/abstain) ===")
# ARC_EDGES = [
#     "delete all my emails",
#     "change my account password",
# ]
# for p in ARC_EDGES:
#     try:
#         run_arc_strict(p)
#         print("UNEXPECTED OK:", p)
#     except AssertionError as e:
#         print("Edge blocked (as expected):", p, "|", str(e).splitlines()[0])
